In [5]:
import sqlite3
import datetime

# Устанавливаем соединение с базой данных
connection = sqlite3.connect('rasa_db.db')
cursor = connection.cursor()

# Получаем все записи с датами
cursor.execute('SELECT id_appointment, date FROM appointment')
appointments = cursor.fetchall()

if appointments:
    # Парсим даты и находим минимальную
    date_objects = []
    for _, date_str in appointments:
        dt = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
        date_objects.append(dt)
    
    min_date = min(date_objects)
    
    # Вычисляем разницу между вчерашней датой и минимальной датой
    yesterday = datetime.datetime.combine(
        datetime.date.today() - datetime.timedelta(days=1),
        min_date.time()  # Сохраняем время из оригинальной минимальной даты
    )
    date_diff = yesterday - min_date
    
    # Обновляем все даты
    for id_appointment, date_str in appointments:
        old_dt = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
        new_dt = old_dt + date_diff
        new_date_str = new_dt.strftime('%Y-%m-%d %H:%M:%S')
        
        cursor.execute(
            'UPDATE appointment SET date = ? WHERE id_appointment = ?',
            (new_date_str, id_appointment)
        )
    
    connection.commit()
    print(f"Обновлено {len(appointments)} записей. Новая минимальная дата: {yesterday.strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print("Нет записей для обновления")

connection.close()

Обновлено 2032 записей. Новая минимальная дата: 2025-07-09 10:00:00
